In [12]:
import numpy as np
import pandas as pd

In [13]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot

# Использование cufflinks в офлайн-режиме
import cufflinks
cufflinks.go_offline()

# Настройка глобальной темы cufflinks
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

In [14]:
filename = 'Массив чеков_ver3.csv'

Для начала прочитаем только заголовки датасета:

In [15]:
headers = pd.read_csv(filename, nrows=0, encoding='utf-16').columns.tolist()
headers

['client_NT',
 'text_kktAddress',
 'Идентификатор покупателя',
 'dateTime',
 'receipt_id',
 'CLEAN_SKU',
 'price_items',
 'quantity_items',
 'sum_items',
 'totalSum',
 'RIGHTHOLDER_NT',
 'BRAND_NT',
 'CLASS_NT',
 'GROUP_NT',
 'CATEGORY_NT',
 'SUBCATEGORY_NT',
 'BASE_NT',
 'lat',
 'lon']

Можно сформировать отдельный список всех магазинов из датасета:

In [58]:
# Сформируем список магазинов из csv-файла, прочитав только колонки 'client_NT', 'text_kktAddress', 'lat', 'lon'
stores = pd.read_csv(filename, usecols=['client_NT', 'text_kktAddress', 'lat', 'lon'], encoding='utf-16').drop_duplicates()
stores

,client_NT,text_kktAddress,lat,lon
0,Окей,"г Москва, ул Озёрная, д 50","55,6647196","37,4372193"
7,Окей,"г Москва, пр-кт Мира, д 211 к 2","55,845855","37,662093"
10,Гиперглобус,"г Москва, ш Новоухтомское, д 2А","55,7128634","37,8454195"
43,Окей,"г Москва, ул Кировоградская, д 13А","55,6121172","37,6070486"
58,Гиперглобус,"г Москва, Сосенское п, п Коммунарка, ул Алекса...","55,557266","37,494368"
109,Окей,"г Москва, ш Каширское, д 14","55,664456","37,627059"
117,Гиперглобус,"г Москва, км Новорижское шоссе 22-й, влд 1 стр 1","55,801981","37,300674"
193,Гиперглобус,"г Москва, км Киевское шоссе 23-й (п Московский...","55,623788","37,422027"
240,Окей,"г Москва, ш Дмитровское, влд 163А к 1","55,90968","37,540721"
250,Окей,"г Москва, ш Головинское, д 5 к 1","55,84016","37,4915995"


In [66]:
df = pd.DataFrame(stores)
df['lat'] = df['lat'].str.replace(',', '.').astype(float)
df['lon'] = df['lon'].str.replace(',', '.').astype(float)

# Строим карту
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    hover_name='client_NT',
    hover_data={'text_kktAddress': True},
    zoom=10,
    height=600,
    title='Расположение магазинов'
)

# Настройки карты (используем mapbox вместо scatter_map)
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":40,"l":0,"b":0},
    mapbox=dict(
        center=dict(lat=df['lat'].mean(), lon=df['lon'].mean()),
        zoom=10
    )
)

# Настройки маркеров (правильный синтаксис)
fig.update_traces(
    marker=dict(
        size=14,        # Размер маркера
        color='red',    # Цвет
        opacity=0.8     # Прозрачность
    )
)

fig.show()

C:\Temp\ipykernel_32464\1226229930.py:6: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



И отдельный список торговых сетей:

In [68]:
trades = stores['client_NT'].drop_duplicates()
trades

0               Окей
10       Гиперглобус
26345      Metro c&c
Name: client_NT, dtype: object

In [69]:
# Сохраним результат в csv файл
stores.sort_values(['client_NT', 'text_kktAddress']).to_csv('Список магазинов.csv', index=False, encoding='utf-8')
stores

,client_NT,text_kktAddress,lat,lon
0,Окей,"г Москва, ул Озёрная, д 50","55,6647196","37,4372193"
7,Окей,"г Москва, пр-кт Мира, д 211 к 2","55,845855","37,662093"
10,Гиперглобус,"г Москва, ш Новоухтомское, д 2А","55,7128634","37,8454195"
43,Окей,"г Москва, ул Кировоградская, д 13А","55,6121172","37,6070486"
58,Гиперглобус,"г Москва, Сосенское п, п Коммунарка, ул Алекса...","55,557266","37,494368"
109,Окей,"г Москва, ш Каширское, д 14","55,664456","37,627059"
117,Гиперглобус,"г Москва, км Новорижское шоссе 22-й, влд 1 стр 1","55,801981","37,300674"
193,Гиперглобус,"г Москва, км Киевское шоссе 23-й (п Московский...","55,623788","37,422027"
240,Окей,"г Москва, ш Дмитровское, влд 163А к 1","55,90968","37,540721"
250,Окей,"г Москва, ш Головинское, д 5 к 1","55,84016","37,4915995"


Переопределим некоторые типы данных для экономии места:

In [19]:
# Здесь сделано под ver2, но под ver3 тоже подходит, так как в ver3 меньше столбцов, а остальные названы также
dtype_spec = {
    'region': 'category',
    'userInn': 'int64',
    'client_NT': 'category',
    'text_kktAddress': 'category',
    'Идентификатор покупателя': 'str',
    'dateTime': 'str', # после преобразуем в datetime
    'Year': 'int16',
    'Month': 'int16',
    'receipt_id': 'str',
    'CLEAN_SKU': 'str',
    'price_items': 'float32',
    'quantity_items': 'float32',
    'sum_items': 'float32',
    'totalSum': 'float32',
    'RIGHTHOLDER_NT': 'str',
    'BRAND_NT': 'category',
    'CLASS_NT': 'category',
    'GROUP_NT': 'category',
    'CATEGORY_NT': 'category',
    'SUBCATEGORY_NT': 'category',
    'BASE_NT': 'category',
    'lat': 'float32',
    'lon': 'float32'
}

Исходя из примера массива, повторяющиеся строки являются частью одного чека. Значит дубликаты строк не убираем

Загружаем датасет:

In [20]:
data = pd.read_csv(filename, encoding='utf-16', dtype=dtype_spec, decimal=',').reset_index()

Если появляется ошибка памяти, то можно загрузить датасет частями:

In [21]:
# data = pd.DataFrame()
# chunk_size = 1000000
# for chunk in pd.read_csv('Массив чеков_ver2.csv', encoding='utf-16', dtype=dtype_spec, decimal=',', chunksize=chunk_size):
#     data = pd.concat([data, chunk], ignore_index=True)
# data  = data.reset_index()

Посмотрим первые 5 строк, чтобы убедиться, что всё подгрузилось правильно:

In [22]:
data.head(5)

,index,client_NT,text_kktAddress,Идентификатор покупателя,dateTime,receipt_id,CLEAN_SKU,price_items,quantity_items,sum_items,totalSum,RIGHTHOLDER_NT,BRAND_NT,CLASS_NT,GROUP_NT,CATEGORY_NT,SUBCATEGORY_NT,BASE_NT,lat,lon
0,0,Окей,"г Москва, ул Озёрная, д 50",NaN,26.06.2022 13:51:00,9960440300975081:17128,Бананы кг,70.38,0.59,41.81,2135.82,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44
1,1,Окей,"г Москва, ул Озёрная, д 50",NaN,26.06.2022 13:46:00,9960440300975081:17126,Зажарка Maggi 60г,33.42,1.00,33.42,3920.33,NESTLE,MAGGI,БАКЛ,БАКАЛЕЯ СЫПУЧАЯ,ПРИПРАВЫ ДЛЯ ПЕРВЫХ И ВТОРЫХ БЛЮД,ПРИПРАВА ДЛЯ ЖАРКОЕ,ПРИПРАВА ДЛЯ ЖАРКОЕ,55.66,37.44
2,2,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бананы кг,96.24,0.91,87.58,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44
3,3,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Яблоки кг,87.49,1.30,113.74,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,ЯБЛОКИ,ЯБЛОКО,55.66,37.44
4,4,Окей,"г Москва, ул Озёрная, д 50",NaN,04.06.2022 18:19:00,9960440301010969:5365,Апельсины кг,109.99,1.69,186.10,13641.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.66,37.44


Проверим данные, например, чек номер `9960440300975081:11611`:

In [23]:
check = data[data['receipt_id'] == '9960440300975081:11611']
check

,index,client_NT,text_kktAddress,Идентификатор покупателя,dateTime,receipt_id,CLEAN_SKU,price_items,quantity_items,sum_items,totalSum,RIGHTHOLDER_NT,BRAND_NT,CLASS_NT,GROUP_NT,CATEGORY_NT,SUBCATEGORY_NT,BASE_NT,lat,lon
2,2,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бананы кг,96.24,0.91,87.58,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44
3,3,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Яблоки кг,87.49,1.30,113.74,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,ЯБЛОКИ,ЯБЛОКО,55.66,37.44
1342543,1342543,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,"Масло льняное Диал-Экспорт нераф 0,5л",182.88,1.00,182.88,2387.77,не определено,DIAL-EXPORT,БАКЛ,РАСТИТЕЛЬНЫЕ МАСЛА,МАСЛО РАСТИТЕЛЬНОЕ,МАСЛО ЛЬНЯНОЕ,МАСЛО ЛЬНЯНОЕ,55.66,37.44
1342544,1342544,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Яблоки Гольден фас пакет кг,102.84,1.35,138.83,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,ЯБЛОКИ,ЯБЛОКО,55.66,37.44
1342545,1342545,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Кофе ТЧН! Растворимый сублимированный 90г,87.49,1.00,87.49,2387.77,СТМ_ОКЕЙ,"ТО, ЧТО НАДО!",ЧКФК,"ЧАЙ, КОФЕ",КОФЕ,КОФЕ РАСТВОРИМЫЙ,КОФЕ РАСТВОРИМЫЙ,55.66,37.44
1342546,1342546,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бумага туалетная ТЧН! 1слойная/1рулон/вторсырье,10.49,1.00,10.49,2387.77,СТМ_ОКЕЙ,"ТО, ЧТО НАДО!",КЗБХ,УХОД И ГИГИЕНА,ТУАЛЕТНАЯ БУМАГА,ТУАЛЕТНАЯ БУМАГА,БУМАГА ТУАЛЕТНАЯ,55.66,37.44
1342547,1342547,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бумага туалетная ТЧН! 1слойная/1рулон/вторсырье,10.49,1.00,10.49,2387.77,СТМ_ОКЕЙ,"ТО, ЧТО НАДО!",КЗБХ,УХОД И ГИГИЕНА,ТУАЛЕТНАЯ БУМАГА,ТУАЛЕТНАЯ БУМАГА,БУМАГА ТУАЛЕТНАЯ,55.66,37.44
1342548,1342548,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бумага туалетная ТЧН! 1слойная/1рулон/вторсырье,10.49,1.00,10.49,2387.77,СТМ_ОКЕЙ,"ТО, ЧТО НАДО!",КЗБХ,УХОД И ГИГИЕНА,ТУАЛЕТНАЯ БУМАГА,ТУАЛЕТНАЯ БУМАГА,БУМАГА ТУАЛЕТНАЯ,55.66,37.44
1342549,1342549,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Макароны Макфа паутинка 450г,60.76,1.00,60.76,2387.77,не определено,MAKFA,БАКЛ,БАКАЛЕЯ СЫПУЧАЯ,МАКАРОННЫЕ ИЗДЕЛИЯ,МАКАРОНЫ ФИГУРНЫЕ,МАКАРОНЫ,55.66,37.44
1342550,1342550,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Хлопья овсяные Русский продукт Геркулес Монаст...,60.77,1.00,60.77,2387.77,РУССКИЕ ПРОДУКТЫ ТОРГ,РУССКИЙ ПРОДУКТ,БАКЛ,БАКАЛЕЯ СЫПУЧАЯ,ХЛОПЬЯ,ХЛОПЬЯ ОВСЯНЫЕ,ХЛОПЬЯ ОВСЯНЫЕ,55.66,37.44


Проверим, что сумма по столбцу `sum_items` равна значению `totalSum`:

In [24]:
check['sum_items'].sum()

np.float32(2387.77)

Посмотрим общую информацию о датафрейме:

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400099 entries, 0 to 2400098
Data columns (total 20 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   index                     int64   
 1   client_NT                 category
 2   text_kktAddress           category
 3   Идентификатор покупателя  object  
 4   dateTime                  object  
 5   receipt_id                object  
 6   CLEAN_SKU                 object  
 7   price_items               float32 
 8   quantity_items            float32 
 9   sum_items                 float32 
 10  totalSum                  float32 
 11  RIGHTHOLDER_NT            object  
 12  BRAND_NT                  category
 13  CLASS_NT                  category
 14  GROUP_NT                  category
 15  CATEGORY_NT               category
 16  SUBCATEGORY_NT            category
 17  BASE_NT                   category
 18  lat                       float32 
 19  lon                       float32 
dtypes:

Посмотрим описательные статистики для числовых столбцов:

In [26]:
data.describe()

,index,price_items,quantity_items,sum_items,totalSum,lat,lon
count,2400099.00,2400099.00,2400099.00,2400099.00,2400099.00,2400099.00,2400099.00
mean,1200049.00,175.96,1.33,178.95,4624.64,55.73,37.52
std,692849.05,238.92,1.82,260.98,3830.94,0.12,0.17
min,0.00,0.00,0.00,0.00,0.00,55.56,37.30
25%,600024.50,59.39,1.00,65.69,2309.50,55.62,37.42
50%,1200049.00,102.51,1.00,115.00,3680.91,55.71,37.49
75%,1800073.50,199.00,1.00,209.98,5748.90,55.84,37.64
max,2400098.00,38986.09,400.00,40499.00,122134.65,55.93,37.88


Посмотрим корреляционные отношения между числовыми столбцами

In [27]:
corr_matrix = data[['price_items', 'quantity_items', 'sum_items', 'totalSum', 'lat', 'lon']].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    zmin=-1, 
    zmax=1, 
    colorscale='RdBu', 
    text=np.round(corr_matrix.values, 2), 
    texttemplate="%{text}",
    hoverinfo="x+y+z",
    showscale=True 
))

# Настройки макета
fig.update_layout(
    title='Корреляционная матрица (интерактивная)',
    xaxis_title='Переменные',
    yaxis_title='Переменные',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed',  # реверсируем ось Y как в sns
    width=800,
    height=600
)

# Дополнительные настройки аннотаций
fig.update_traces(
    textfont_size=12,
    textfont_color='black'
)

fig.show()

Практически везде слабая корреляционная зависимость, за исключением зависимости sum_items - price_items, она средняя(0.62).

Добавим новые столбцы hour и weekday. И переименуем столбец "Идентификатор покупателя" в customer_id

In [28]:
data['hour'] = pd.to_datetime(data.dateTime).dt.hour

weekdays = {
    0: '1.Понедельник',
    1: '2.Вторник',
    2: '3.Среда',
    3: '4.Четверг',
    4: '5.Пятница',
    5: '6.Суббота',
    6: '7.Воскресенье'
    }
data['weekday'] = pd.to_datetime(data.dateTime).dt.weekday.map(weekdays)

data = data.rename(columns={'Идентификатор покупателя': 'customer_id'})

C:\Temp\ipykernel_32464\2744415287.py:1: UserWarning:

Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

C:\Temp\ipykernel_32464\2744415287.py:12: UserWarning:

Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



Проведём `ABC-анализ` для каждой торговой сети.

Классифицируем товары `CLEAN_SKU` по прибыли и количеству проданных позиций. А так же по `CATEGORY_NT` по количеству проданных позиций

In [29]:
def perform_abc(df, index):
    cols = list(df.columns)
    for i in index:
        cols.remove(i)
    groupped_df = df.groupby(index[1]).agg({col: 'sum' for col in cols})
    for col in cols:
        groupped_df[f'rel_{col}'] = groupped_df[col] / sum(groupped_df[col])
        groupped_df = groupped_df.sort_values(f'rel_{col}', ascending=False)
        groupped_df[f'cumsum_{col}'] = groupped_df[f'rel_{col}'].cumsum()
        groupped_df[f'abc_{col}'] = np.where(groupped_df[f'cumsum_{col}'] < 0.8, 'A', np.where(groupped_df[f'cumsum_{col}'] < 0.95, 'B', 'C'))
    return groupped_df[cols + [f'abc_{col}' for col in cols]]

In [30]:
abc_all_SKU = data[['client_NT', 'CLEAN_SKU', 'price_items', 'quantity_items']].dropna()
abc_all_SKU

,client_NT,CLEAN_SKU,price_items,quantity_items
0,Окей,Бананы кг,70.38,0.59
1,Окей,Зажарка Maggi 60г,33.42,1.00
2,Окей,Бананы кг,96.24,0.91
3,Окей,Яблоки кг,87.49,1.30
4,Окей,Апельсины кг,109.99,1.69
...,...,...,...,...
2400094,Окей,Кондиционер конц д/белья Vernel supreme elegan...,199.99,1.00
2400095,Окей,Абрикосы 500г упак,169.99,1.00
2400096,Окей,Пакет подарочный Арт и Дизайн ST премиум 12х36см,102.99,1.00
2400097,Окей,"Вино Вадиаперти Греко Ди Туфо бел. сух. 4,5-16...",1099.02,1.00


In [31]:
abc_SKU = pd.DataFrame()
for trade in trades:
    tmp = pd.DataFrame()
    tmp = perform_abc(abc_all_SKU[abc_all_SKU['client_NT'] == trade], ['client_NT', 'CLEAN_SKU']).reset_index()
    tmp['client_NT'] = trade
    abc_SKU = pd.concat([abc_SKU, tmp], ignore_index=True).dropna()
abc_SKU

,CLEAN_SKU,price_items,quantity_items,abc_price_items,abc_quantity_items,client_NT
0,Пакет из переработанного материала Kinglet 40х...,67268.21,9074.00,A,A,Окей
1,Бананы кг,652318.31,8526.12,A,A,Окей
2,"БЗМЖ Молоко утп Домик в деревне 3,2% 925мл тба",312110.16,7176.00,A,A,Окей
3,Картофель ранний импортный кг,252049.86,6234.26,A,A,Окей
4,"Сахар-песок бел. кристал. ГОСТ 33222-2015, кат...",199892.50,5130.00,A,A,Окей
...,...,...,...,...,...,...
71228,630Г КОЛБАСА ОКРАИНА ЗЕРНИСТАЯ С/К,565.59,0.34,A,C,Metro c&c
71229,САЛАТ АЙСБЕРГ,159.01,0.31,B,C,Metro c&c
71230,ЧЕСНОК,196.96,0.31,B,C,Metro c&c
71231,ВИШНЯ,239.00,0.21,B,C,Metro c&c


In [32]:
abc_all_CATEGORY = data[['client_NT', 'CATEGORY_NT', 'quantity_items']].dropna()
abc_all_CATEGORY

,client_NT,CATEGORY_NT,quantity_items
0,Окей,ФРУКТЫ ОСНОВНЫЕ,0.59
1,Окей,ПРИПРАВЫ ДЛЯ ПЕРВЫХ И ВТОРЫХ БЛЮД,1.00
2,Окей,ФРУКТЫ ОСНОВНЫЕ,0.91
3,Окей,ФРУКТЫ ОСНОВНЫЕ,1.30
5,Окей,ОВОЩИ РАЗНЫЕ,0.41
...,...,...,...
2400094,Окей,СР-ВА ДЛЯ СТИРКИ,1.00
2400095,Окей,ФРУКТЫ ОСНОВНЫЕ,1.00
2400096,Окей,ВСЕ ДЛЯ ВЕСЕЛОГО ПРАЗДНИКА,1.00
2400097,Окей,ВИНО,1.00


In [33]:
abc_CATEGORY = pd.DataFrame()
for trade in trades:
    tmp = pd.DataFrame()
    tmp = perform_abc(abc_all_CATEGORY[abc_all_CATEGORY['client_NT'] == trade], ['client_NT', 'CATEGORY_NT']).reset_index()
    tmp['client_NT'] = trade
    abc_CATEGORY = pd.concat([abc_CATEGORY, tmp], ignore_index=True).dropna()
abc_CATEGORY = abc_CATEGORY.rename(columns={'abc_quantity_items': 'abc_quantity_items_CAT'})
abc_CATEGORY


C:\Temp\ipykernel_32464\3440270023.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Temp\ipykernel_32464\3440270023.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Temp\ipykernel_32464\3440270023.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,CATEGORY_NT,quantity_items,abc_quantity_items_CAT,client_NT
0,"ЗООКОРМ, ЛАКОМСТВА ДЛЯ КОШЕК",51633.00,A,Окей
1,ТВОРОЖНАЯ ГРУППА,29286.00,A,Окей
2,ВОДА,29043.00,A,Окей
3,МОЛОКО,28928.00,A,Окей
4,КОРНЕПЛОДЫ,28415.69,A,Окей
...,...,...,...,...
1279,"ШАРФЫ, ПАЛАНТИНЫ, ПАРЕО",0.00,C,Metro c&c
1280,САКЕ,0.00,C,Metro c&c
1281,БАЛЬЗАМ,0.00,C,Metro c&c
1282,НОСИТЕЛИ ИНФОРМАЦИИ С ЗАПИСЬЮ,0.00,C,Metro c&c


In [34]:
xyz_data = data[['client_NT', 'CLEAN_SKU', 'dateTime', 'quantity_items']]
xyz_data['dateTime'] = pd.to_datetime(xyz_data['dateTime']).dt.strftime('%W') # По неделям
xyz_data

C:\Temp\ipykernel_32464\347484163.py:2: UserWarning:

Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,client_NT,CLEAN_SKU,dateTime,quantity_items
0,Окей,Бананы кг,25,0.59
1,Окей,Зажарка Maggi 60г,25,1.00
2,Окей,Бананы кг,17,0.91
3,Окей,Яблоки кг,17,1.30
4,Окей,Апельсины кг,22,1.69
...,...,...,...,...
2400094,Окей,Кондиционер конц д/белья Vernel supreme elegan...,20,1.00
2400095,Окей,Абрикосы 500г упак,20,1.00
2400096,Окей,Пакет подарочный Арт и Дизайн ST премиум 12х36см,21,1.00
2400097,Окей,"Вино Вадиаперти Греко Ди Туфо бел. сух. 4,5-16...",21,1.00


In [35]:
def perform_xyz(df, date_col, index_cols, value_col):
    # Группировка и агрегация (сумма quantity_items по неделям)
    grouped = df.groupby(index_cols + [date_col])[value_col].sum().reset_index()
    # Фильтрация SKU с достаточным количеством недель (>1)
    sku_week_counts = grouped.groupby(index_cols)[date_col].count().reset_index()
    valid_skus = sku_week_counts[sku_week_counts[date_col] > 1][index_cols]
    filtered = pd.merge(grouped, valid_skus, on=index_cols)
    
    # Расчет коэффициента вариации
    cv_result = (filtered.groupby(index_cols)[value_col]
                 .apply(lambda x: x.std()/x.mean())
                 .reset_index(name='cv'))
    
    # 6. Классификация XYZ
    cv_result['XYZ'] = np.where(cv_result['cv'] < 0.1, 'X', np.where(cv_result['cv'] < 0.25, 'Y', 'Z'))
    
    return cv_result

In [36]:
xyz_SKU = pd.DataFrame()
for trade in trades:
    tmp = pd.DataFrame()
    tmp = perform_xyz(xyz_data[xyz_data['client_NT'] == trade], date_col='dateTime', index_cols=['CLEAN_SKU'], value_col='quantity_items')
    tmp['client_NT'] = trade
    xyz_SKU = pd.concat([xyz_SKU, tmp], ignore_index=True).dropna()
xyz_SKU

,CLEAN_SKU,cv,XYZ,client_NT
0,"""Ассорти """"Черри и корнишоны"""" 500г""",0.00,X,Окей
1,"""Ассорти овощное """"Пятерка"""" 1000г""",0.47,Z,Окей
2,"""БЗМЖ сыр """"Моне"""" с массовой долей жира 45% (...",0.34,Z,Окей
3,"""Бальзам Травяной 250 мл """"Живая природа"""" (Ок...",0.29,Z,Окей
4,"""Батончик мюсли """"О'кей"""" вишня с цельными зла...",0.77,Z,Окей
...,...,...,...,...
52672,ТОМАТЫ ВЕС (КГ),0.81,Z,Metro c&c
52673,ТУШКА ЦБ СТМ MC ПАКЕТ ОХЛ (КГ),0.01,X,Metro c&c
52674,ФИЛЕ КУРИНОЕ METRO CHEF МИНИКОРОБ ОХЛ (КГ),0.05,X,Metro c&c
52675,ЯБЛОКИ ГРЕННИ СМИТ ВЕС (КГ),0.52,Z,Metro c&c


Теперь соединим результаты abc_SKU, abc_CATEGORY, xyz_SKU в один датафрейм и выгрузим в pickle-файл

In [37]:
analysis_df = pd.DataFrame()
analysis_df = pd.merge(data, abc_SKU, on=['client_NT', 'CLEAN_SKU'], how='left')
analysis_df = pd.merge(analysis_df, abc_CATEGORY, on=['client_NT', 'CATEGORY_NT'], how='left')
analysis_df = pd.merge(analysis_df, xyz_SKU, on=['client_NT', 'CLEAN_SKU'], how='left')
analysis_df = analysis_df[['client_NT', 'text_kktAddress', 
                            'customer_id', 'dateTime', 'receipt_id', 'CLEAN_SKU', 'price_items_x',
                            'abc_price_items', 'abc_quantity_items', 'abc_quantity_items_CAT', 'XYZ', 
                            'quantity_items', 'sum_items', 'totalSum',
                            'RIGHTHOLDER_NT', 'BRAND_NT', 'CLASS_NT', 'GROUP_NT', 'CATEGORY_NT',
                            'SUBCATEGORY_NT', 'BASE_NT', 'lat', 'lon']]
analysis_df

,client_NT,text_kktAddress,customer_id,dateTime,receipt_id,CLEAN_SKU,price_items_x,abc_price_items,abc_quantity_items,abc_quantity_items_CAT,...,totalSum,RIGHTHOLDER_NT,BRAND_NT,CLASS_NT,GROUP_NT,CATEGORY_NT,SUBCATEGORY_NT,BASE_NT,lat,lon
0,Окей,"г Москва, ул Озёрная, д 50",NaN,26.06.2022 13:51:00,9960440300975081:17128,Бананы кг,70.38,A,A,A,...,2135.82,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44
1,Окей,"г Москва, ул Озёрная, д 50",NaN,26.06.2022 13:46:00,9960440300975081:17126,Зажарка Maggi 60г,33.42,B,A,A,...,3920.33,NESTLE,MAGGI,БАКЛ,БАКАЛЕЯ СЫПУЧАЯ,ПРИПРАВЫ ДЛЯ ПЕРВЫХ И ВТОРЫХ БЛЮД,ПРИПРАВА ДЛЯ ЖАРКОЕ,ПРИПРАВА ДЛЯ ЖАРКОЕ,55.66,37.44
2,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бананы кг,96.24,A,A,A,...,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44
3,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Яблоки кг,87.49,A,A,A,...,2387.77,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,ЯБЛОКИ,ЯБЛОКО,55.66,37.44
4,Окей,"г Москва, ул Озёрная, д 50",NaN,04.06.2022 18:19:00,9960440301010969:5365,Апельсины кг,109.99,A,A,NaN,...,13641.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.66,37.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400094,Окей,"г Москва, ул Кировоградская, д 13А",NaN,21.05.2022 18:02:00,9960440300963884:53239,Кондиционер конц д/белья Vernel supreme elegan...,199.99,B,B,A,...,818.97,не определено,VERNEL,КЗБХ,БЫТОВАЯ ХИМИЯ,СР-ВА ДЛЯ СТИРКИ,КОНДИЦИОНЕР-ОПОЛАСКИВАТЕЛЬ,КОНДИЦИОНЕР ДЛЯ БЕЛЬЯ,55.61,37.61
2400095,Окей,"г Москва, ул Кировоградская, д 13А",NaN,21.05.2022 18:02:00,9960440300963884:53239,Абрикосы 500г упак,169.99,A,A,A,...,818.97,NaN,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,АБРИКОС,АБРИКОС,55.61,37.61
2400096,Окей,"г Москва, ул Кировоградская, д 13А",8e72f68f3891ce5f91503cb8dbe1b210,26.05.2022 18:08:00,9960440300964629:76254,Пакет подарочный Арт и Дизайн ST премиум 12х36см,102.99,C,C,C,...,1832.00,не определено,АРТ И ДИЗАЙН,КЦИГ,ИГРУШКИ,ВСЕ ДЛЯ ВЕСЕЛОГО ПРАЗДНИКА,ПОДАРОЧНЫЕ ПАКЕТЫ И УПАКОВКА,ПАКЕТ ПОДАРОЧНЫЙ,55.61,37.61
2400097,Окей,"г Москва, ул Кировоградская, д 13А",8e72f68f3891ce5f91503cb8dbe1b210,26.05.2022 18:08:00,9960440300964629:76254,"Вино Вадиаперти Греко Ди Туфо бел. сух. 4,5-16...",1099.02,B,C,B,...,1832.00,NaN,NaN,АЛКГ,ВИНО,ВИНО,ВИНО ТИХОЕ,ВИНО ТИХОЕ,55.61,37.61


In [52]:
# Сохраним результаты ABC|XYZ-анализа в pcl файл
analysis_df.to_pickle('ABC-XYZ.pcl.gz', compression='gzip')
# Можно скачать его https://disk.yandex.ru/d/aVAb0DY98upb3w

In [53]:
# Если надо загрузить, то можно воспользоваться:
# abc_xyz = pd.read_pickle('ABC-XYZ.pcl.gz', compression='gzip')

Сформируем csv-файлы по торговым сетям

In [54]:
for trade in trades:
    analysis_df[analysis_df['client_NT'] == trade].to_csv(f'ABC-XYZ {trade}.csv', index=False)

Сравним цены на одинаковые товары в разных торговых сетях.

Составим сводную таблицу по категориям товара (`CATEGORY_NT`). Столбцами будут торговые сети, а в качестве значений будет средняя цена по категории.

In [55]:
data.pivot_table(
    index='CATEGORY_NT',
    columns='client_NT',
    values='price_items',
    aggfunc='mean', 
    observed=False
).sort_index()

client_NT,Metro c&c,Гиперглобус,Окей
CATEGORY_NT,,,
АКСЕССУАРИКА,NaN,389.48,429.44
АКСЕССУАРЫ,NaN,215.04,228.87
АКСЕССУАРЫ ДЛЯ ОБУВИ,NaN,175.51,210.36
АКСЕССУАРЫ ДЛЯ ПРОГУЛКИ И ДРЕССИРОВКИ ЖИВОТНЫХ,NaN,211.44,258.78
АССОРТИ МЯСНОЙ ГАСТРОНОМИИ,NaN,377.21,151.43
...,...,...,...
"СИГАРЫ, СИГАРИЛЫ",NaN,200.00,NaN
"НАКЛАДКИ, ВКЛАДЫШИ, ПРОКЛАДКИ ДЛЯ БЕРЕМЕННЫХ И КОРМЯЩИХ",NaN,244.00,NaN
МЯСО УТКИ ЗАМОРОЖЕННОЕ,NaN,NaN,346.86


Так же в этой таблице видны категории товаров, которые присутствуют в одной сети, но отсутствуют в другой

Можно посмотреть различие цен на товары, которые присутствуют во всех торговых сетях

In [56]:
data.pivot_table(
    index='CATEGORY_NT',
    columns='client_NT',
    values='price_items',
    aggfunc='mean', 
    observed=False
).sort_index().dropna()

client_NT,Metro c&c,Гиперглобус,Окей
CATEGORY_NT,,,
БАГЕТ,105.85,50.79,27.36
БАРАНОЧНЫЕ ИЗДЕЛИЯ,43.78,55.19,48.15
БАТАРЕЙКИ,259.00,212.60,204.88
БЕЗРЕЦЕПТУРНАЯ АПТЕКА,356.06,219.57,424.06
БЛЮДА ДЛЯ ПЕРЕКУСА,195.80,198.76,181.43
...,...,...,...
ШОКОЛАДНЫЕ ЯЙЦА,88.26,108.87,93.64
ЯГОДЫ,204.84,302.72,194.01
ЯЙЦА ПРОЧИЕ,349.07,163.12,111.38


In [57]:
data

,index,client_NT,text_kktAddress,customer_id,dateTime,receipt_id,CLEAN_SKU,price_items,quantity_items,sum_items,...,BRAND_NT,CLASS_NT,GROUP_NT,CATEGORY_NT,SUBCATEGORY_NT,BASE_NT,lat,lon,hour,weekday
0,0,Окей,"г Москва, ул Озёрная, д 50",NaN,26.06.2022 13:51:00,9960440300975081:17128,Бананы кг,70.38,0.59,41.81,...,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44,13,7.Воскресенье
1,1,Окей,"г Москва, ул Озёрная, д 50",NaN,26.06.2022 13:46:00,9960440300975081:17126,Зажарка Maggi 60г,33.42,1.00,33.42,...,MAGGI,БАКЛ,БАКАЛЕЯ СЫПУЧАЯ,ПРИПРАВЫ ДЛЯ ПЕРВЫХ И ВТОРЫХ БЛЮД,ПРИПРАВА ДЛЯ ЖАРКОЕ,ПРИПРАВА ДЛЯ ЖАРКОЕ,55.66,37.44,13,7.Воскресенье
2,2,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Бананы кг,96.24,0.91,87.58,...,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,БАНАНЫ,БАНАН,55.66,37.44,13,7.Воскресенье
3,3,Окей,"г Москва, ул Озёрная, д 50",NaN,01.05.2022 13:26:00,9960440300975081:11611,Яблоки кг,87.49,1.30,113.74,...,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,ЯБЛОКИ,ЯБЛОКО,55.66,37.44,13,7.Воскресенье
4,4,Окей,"г Москва, ул Озёрная, д 50",NaN,04.06.2022 18:19:00,9960440301010969:5365,Апельсины кг,109.99,1.69,186.10,...,NaN,NaN,NaN,NaN,NaN,NaN,55.66,37.44,18,6.Суббота
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400094,2400094,Окей,"г Москва, ул Кировоградская, д 13А",NaN,21.05.2022 18:02:00,9960440300963884:53239,Кондиционер конц д/белья Vernel supreme elegan...,199.99,1.00,199.99,...,VERNEL,КЗБХ,БЫТОВАЯ ХИМИЯ,СР-ВА ДЛЯ СТИРКИ,КОНДИЦИОНЕР-ОПОЛАСКИВАТЕЛЬ,КОНДИЦИОНЕР ДЛЯ БЕЛЬЯ,55.61,37.61,18,6.Суббота
2400095,2400095,Окей,"г Москва, ул Кировоградская, д 13А",NaN,21.05.2022 18:02:00,9960440300963884:53239,Абрикосы 500г упак,169.99,1.00,169.99,...,NaN,ФРОВ,СВЕЖИЕ ФРУКТЫ,ФРУКТЫ ОСНОВНЫЕ,АБРИКОС,АБРИКОС,55.61,37.61,18,6.Суббота
2400096,2400096,Окей,"г Москва, ул Кировоградская, д 13А",8e72f68f3891ce5f91503cb8dbe1b210,26.05.2022 18:08:00,9960440300964629:76254,Пакет подарочный Арт и Дизайн ST премиум 12х36см,102.99,1.00,102.99,...,АРТ И ДИЗАЙН,КЦИГ,ИГРУШКИ,ВСЕ ДЛЯ ВЕСЕЛОГО ПРАЗДНИКА,ПОДАРОЧНЫЕ ПАКЕТЫ И УПАКОВКА,ПАКЕТ ПОДАРОЧНЫЙ,55.61,37.61,18,4.Четверг
2400097,2400097,Окей,"г Москва, ул Кировоградская, д 13А",8e72f68f3891ce5f91503cb8dbe1b210,26.05.2022 18:08:00,9960440300964629:76254,"Вино Вадиаперти Греко Ди Туфо бел. сух. 4,5-16...",1099.02,1.00,1099.02,...,NaN,АЛКГ,ВИНО,ВИНО,ВИНО ТИХОЕ,ВИНО ТИХОЕ,55.61,37.61,18,4.Четверг


Подготовим таблицу для среднего чека

In [38]:
mean_check = data.groupby('receipt_id').agg({
    'totalSum': 'max',
    'client_NT': 'first',
    'text_kktAddress': 'first',
    'hour': 'first',
    'weekday': 'first'
    
}).reset_index()

Средний чек в каждой сети

In [40]:
mc1 = mean_check.pivot_table(
    columns='client_NT',
    values='totalSum',
    aggfunc='mean', 
    observed=False
)
mc1

client_NT,Metro c&c,Гиперглобус,Окей
totalSum,8730.40,3360.11,2496.69


Сформируем таблицу со средним чеком по часам во всех торговых сетях для сравнения:

In [41]:
mc2 =mean_check.pivot_table(
    values='totalSum',
    index='hour', 
    columns='client_NT', 
    aggfunc='mean', 
    observed=False
    )
mc2

client_NT,Metro c&c,Гиперглобус,Окей
hour,,,
0,NaN,2376.18,2569.96
1,NaN,2623.16,NaN
2,NaN,2407.63,NaN
3,NaN,2262.54,NaN
4,NaN,2684.25,NaN
5,7796.83,2627.15,NaN
6,1856.69,2821.99,NaN
7,23387.27,3177.00,NaN
8,16492.82,3633.36,2417.80


In [56]:
df_long = mc2.reset_index().melt(id_vars='hour', var_name='client_NT', value_name='mean_totalSum')

# Строим barplot
fig = px.bar(
    df_long,
    x='hour',
    y='mean_totalSum',
    color='client_NT',
    barmode='group',  # Группированные столбцы
    title='Средний чек по клиентам по часам',
    labels={
        'hour': 'Час дня',
        'mean_totalSum': 'Средний чек',
        'client_NT': 'Клиент'
    },
    text_auto='.2f'  # Автоматическое отображение значений с 2 знаками после запятой
)

# Настраиваем отображение
fig.update_layout(
    xaxis=dict(
        type='category',  # Чтобы часы отображались как категории
        title='Час дня'
    ),
    yaxis_title='Средний чек',
    legend_title='Торговая сеть',
    hovermode='x unified'  # Общая подсказка для всех столбцов при наведении
)

# Добавляем подписи значений над столбцами
fig.update_traces(
    textfont_size=12,
    textangle=0,
    textposition="outside",
    cliponaxis=False
)

fig.show()

Аналогично сформируем таблицу со средним чеком по дням недели во всех торговых сетях для сравнения:

In [42]:
mc3 = mean_check.pivot_table(
    values='totalSum',
    index='weekday',
    columns='client_NT',
    aggfunc='mean',
    observed=False
    )
mc3

client_NT,Metro c&c,Гиперглобус,Окей
weekday,,,
1.Понедельник,6744.43,3247.52,2327.74
2.Вторник,7085.23,3228.55,2341.79
3.Среда,8142.43,3228.67,2188.79
4.Четверг,9022.47,3314.41,2422.37
5.Пятница,29299.12,3557.94,2616.60
6.Суббота,13894.15,3541.05,2833.78
7.Воскресенье,5931.00,3360.12,2611.26


In [55]:
df_long = mc3.reset_index().melt(id_vars='weekday', var_name='client_NT', value_name='mean_totalSum')

# Строим barplot
fig = px.bar(
    df_long,
    x='weekday',
    y='mean_totalSum',
    color='client_NT',
    barmode='group',  # Группированные столбцы
    title='Средний чек по клиентам по часам',
    labels={
        'weekday': 'День недели',
        'mean_totalSum': 'Средний чек',
        'client_NT': 'Торговая сеть'
    },
    text_auto='.2f'  # Автоматическое отображение значений с 2 знаками после запятой
)

# Настраиваем отображение
fig.update_layout(
    xaxis=dict(
        type='category',  # Чтобы часы отображались как категории
        title='Час дня'
    ),
    yaxis_title='Средний чек',
    legend_title='Клиенты',
    hovermode='x unified'  # Общая подсказка для всех столбцов при наведении
)

# Добавляем подписи значений над столбцами
fig.update_traces(
    textfont_size=12,
    textangle=0,
    textposition="outside",
    cliponaxis=False
)

fig.show()

Сформируем сводную таблицу по среднему чеку по часам для каждого магазина торговой сети

In [63]:
mean_check.pivot_table(
    values='totalSum',
    index='hour', 
    columns=['client_NT', 'text_kktAddress'], 
    aggfunc='mean', 
    observed=False
    )

client_NT                             Metro c&c                               \
text_kktAddress г Москва, пр-кт Мира, д 211 к 1 г Москва, ул Рябиновая, д 59   
hour                                                                           
0                                           NaN                          NaN   
1                                           NaN                          NaN   
2                                           NaN                          NaN   
3                                           NaN                          NaN   
4                                           NaN                          NaN   
5                                           NaN                          NaN   
6                                           NaN                       582.90   
7                                           NaN                          NaN   
8                                      30184.29                      4936.70   
9                                      26418.88                          NaN   
10                                      9058.55                      6156.73   
11                                          NaN                      1050.39   
12                                          NaN                      6699.00   
13                                     37413.46                      7125.34   
14                                     30164.58                      4478.73   
15                                          NaN                      5967.40   
16                                          NaN                      4864.25   
17                                          NaN                      4460.68   
18                                          NaN                      5103.12   
19                                          NaN                      4087.15   
20                                          NaN                      4287.85   
21                                          NaN                      7707.20   
22                                          NaN                      1791.16   
23                                          NaN                      3892.53   

client_NT                                       \
text_kktAddress г Москва, ул Дорожная, д 1 к 1   
hour                                             
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                     54782.99   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                         NaN   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   
19                                         NaN   
20                                         NaN   
21                                         NaN   
22                                         NaN   
23                                         NaN   

client_NT                                                                      \
text_kktAddress г Москва, ш Ленинградское, д 71Г г Москва, км МКАД 104-й, д 6   
hour                                                                            
0                                            NaN                          NaN   
1                                            NaN                          NaN   
2                                            NaN                          NaN   
3 

Сформируем сводную таблицу по среднему чеку по дням недели для каждого магазина торговой сети

In [62]:
mean_check.pivot_table(
    values='totalSum',
    index='weekday', 
    columns=['client_NT', 'text_kktAddress'], 
    aggfunc='mean', 
    observed=False
    )

client_NT                             Metro c&c                               \
text_kktAddress г Москва, пр-кт Мира, д 211 к 1 г Москва, ул Рябиновая, д 59   
weekday                                                                        
1.Понедельник                           9058.55                      2372.96   
2.Вторник                              26418.88                      6155.26   
3.Среда                                30164.58                      4819.15   
4.Четверг                                   NaN                      5571.26   
5.Пятница                              33798.88                          NaN   
6.Суббота                                   NaN                          NaN   
7.Воскресенье                               NaN                          NaN   

client_NT                                       \
text_kktAddress г Москва, ул Дорожная, д 1 к 1   
weekday                                          
1.Понедельник                              NaN   
2.Вторник                                  NaN   
3.Среда                                    NaN   
4.Четверг                             54782.99   
5.Пятница                                  NaN   
6.Суббота                                  NaN   
7.Воскресенье                              NaN   

client_NT                                                                      \
text_kktAddress г Москва, ш Ленинградское, д 71Г г Москва, км МКАД 104-й, д 6   
weekday                                                                         
1.Понедельник                           10730.21                          NaN   
2.Вторник                                6668.59                          NaN   
3.Среда                                  9358.86                          NaN   
4.Четверг                                7434.91                          NaN   
5.Пятница                                    NaN                     28396.60   
6.Суббота                                9929.50                          NaN   
7.Воскресенье                            5931.00                          NaN   

client_NT                                                   \
text_kktAddress г Москва, Московский п, кв-л 34, д 3 стр 1   
weekday                                                      
1.Понедельник                                          NaN   
2.Вторник                                         16060.34   
3.Среда                                                NaN   
4.Четверг                                         21240.77   
5.Пятница                                              NaN   
6.Суббота                                         25788.10   
7.Воскресенье                                          NaN   

client_NT                                              \
text_kktAddress г Москва, ул Маршала Прошлякова, д 14   
weekday                                                 
1.Понедельник                                     NaN   
2.Вторник                                         NaN   
3.Среда                                           NaN   
4.Четверг                                         NaN   
5.Пятница                                    18809.88   
6.Суббота                                         NaN   
7.Воскресенье                                     NaN   

client_NT                                        \
text_kktAddress г Москва, ш Дмитровское, д 165Б   
weekday                                           
1.Понедельник                               NaN   
2.Вторник                                   NaN   
3.Среда                                     NaN   
4.Четверг                                   NaN   
5.Пятница                              31691.36   
6.Суббота                                   NaN   
7.Воскресенье                               NaN   

client_NT                                                                    Гиперглобус  \
text_kktAddress г Москва, Сосенское п, п Коммунарка, ул Александры Монаховой, д 61 стр 1   


Проведём `RFM-анализ`

Сформируем датафрейм, в котором `customer_id` не `Null`. Кроме того, возмьём лишь по одной строке `receipt_id`, так как нам необходимо узнать количество посещений

In [70]:
data_rfm = data[~data['customer_id'].isnull()] # Оставим только не нулевые `cusomer_id`
data_rfm = data_rfm.groupby('receipt_id').first() # Достаточно взять только первую строку чека
data_rfm['dateTime'] = pd.to_datetime(data_rfm['dateTime'], dayfirst=True) # Преобразуем `dateTime` в формат datetime, если вдруг он текстовый
data_rfm = data_rfm.reset_index()
snapshot_date = data_rfm['dateTime'].max() + pd.Timedelta(days=1) # Дата "снапшота" - следующий день после максимальной даты в датафрейме

In [71]:
rfm = data_rfm.groupby(['customer_id'], observed=False).agg({
    'client_NT': 'first',
    'dateTime': lambda x: (snapshot_date - pd.to_datetime(x, dayfirst=True).max()).days,
    'receipt_id': 'nunique',
    'totalSum': 'sum',  
}).rename(columns={
    'dateTime': 'Recency',      # (дней с последней покупки)
    'receipt_id': 'Frequency',  # (число уникальных чеков)
    'totalSum': 'Monetary'      # (общая сумма покупок)
});
rfm

,client_NT,Recency,Frequency,Monetary
customer_id,,,,
000210750313104b1dcde90697a23cb1,Окей,53,1,1249.81
000c4642b320702ed33605722ec4bab3,Гиперглобус,25,1,4725.13
001d4e7abf6108c688d943ce99d7fc74,Гиперглобус,47,1,710.95
0023a42d6f0698d6fee2ad95121eecc3,Гиперглобус,10,2,499.64
0027021d451a8c17af41e950fe450f60,Окей,25,2,1266.14
...,...,...,...,...
ffdf60acbcda04db2e1d50b17432303e,Гиперглобус,57,4,6666.19
ffe6d0b5b97459e17d775c15258bd40e,Окей,4,1,1223.09
ffe6f731458ba86281ea71b474388cc8,Гиперглобус,22,1,5573.59


In [72]:
# Зададим квантили
q = [0.33, 0.66]
qr = rfm['Recency'].quantile(q).tolist()
qf = rfm['Frequency'].quantile(q).tolist()
qm = rfm['Monetary'].quantile(q).tolist()

In [73]:
# Функция для присвоения баллов (1-3)
def rfm_score(x, q, metric=False):
    if metric == 'Recency':  # Чем меньше дней, тем лучше
        if x <= q[0]: return 3
        elif x <= q[1]: return 2
        else: return 1
    else:  # Frequency и Monetary - чем больше, тем лучше
        if x <= q[0]: return 1
        elif x <= q[1]: return 2
        else: return 3

# Применяем scoring
rfm['R'] = rfm['Recency'].apply(rfm_score, q=qr, metric='Recency')
rfm['F'] = rfm['Frequency'].apply(rfm_score, q=qf)
rfm['M'] = rfm['Monetary'].apply(rfm_score, q=qm)

In [74]:
rfm

,client_NT,Recency,Frequency,Monetary,R,F,M
customer_id,,,,,,,
000210750313104b1dcde90697a23cb1,Окей,53,1,1249.81,1,1,1
000c4642b320702ed33605722ec4bab3,Гиперглобус,25,1,4725.13,2,1,2
001d4e7abf6108c688d943ce99d7fc74,Гиперглобус,47,1,710.95,1,1,1
0023a42d6f0698d6fee2ad95121eecc3,Гиперглобус,10,2,499.64,3,2,1
0027021d451a8c17af41e950fe450f60,Окей,25,2,1266.14,2,2,1
...,...,...,...,...,...,...,...
ffdf60acbcda04db2e1d50b17432303e,Гиперглобус,57,4,6666.19,1,3,3
ffe6d0b5b97459e17d775c15258bd40e,Окей,4,1,1223.09,3,1,1
ffe6f731458ba86281ea71b474388cc8,Гиперглобус,22,1,5573.59,2,1,2


In [75]:
# Комбинированный RFM-скоринг
rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm

,client_NT,Recency,Frequency,Monetary,R,F,M,RFM_Score
customer_id,,,,,,,,
000210750313104b1dcde90697a23cb1,Окей,53,1,1249.81,1,1,1,111
000c4642b320702ed33605722ec4bab3,Гиперглобус,25,1,4725.13,2,1,2,212
001d4e7abf6108c688d943ce99d7fc74,Гиперглобус,47,1,710.95,1,1,1,111
0023a42d6f0698d6fee2ad95121eecc3,Гиперглобус,10,2,499.64,3,2,1,321
0027021d451a8c17af41e950fe450f60,Окей,25,2,1266.14,2,2,1,221
...,...,...,...,...,...,...,...,...
ffdf60acbcda04db2e1d50b17432303e,Гиперглобус,57,4,6666.19,1,3,3,133
ffe6d0b5b97459e17d775c15258bd40e,Окей,4,1,1223.09,3,1,1,311
ffe6f731458ba86281ea71b474388cc8,Гиперглобус,22,1,5573.59,2,1,2,212


Сохраним результаты в csv-файлы

In [80]:
for trade in trades:
    rfm[rfm['client_NT'] == trade].to_csv(f'RFM {trade}.csv', index=True)

Выводы:

- Применённые функции для проведения ABC, XYZ и RFM анализов можно, при необходимости, использовать для каждого отдельного магазина, сформировав отфильтрованный датафрейм.
- ABC-анализ говорит, что товары категории:
    - A - самые важные. Это 20% товаров, которые приносят 80% выручки. Они всегда должны быть на складе. Дефицит этих товаров приводит к проседанию по выручке
    - B - требуют контроля, так как это товары-кандидаты в категорию A. Они не приносят много выручки, но лучше, чтобы они были в наличии.
    - C - товары, которые не приносят значимой выручки, их можно реже закупать и не хранить на складах. Некоторые из них можно даже исключить из ассортимента.
- XYZ-анализ говорит, что товары категории:
    - X - товары со стабильным спросом, они всегда должны быть в наличии.
    - Y - товары со средним спросом, возможно, сезонные. Закупать в большом количестве смысла нет.
    - Z - товары с редким спросом. Необходимо проанализировать поставки, если с поставками проблем не было (то есть товара было достаточно), то от таких товаров можно либо отказаться, либо поставлять по предзаказу.
- RFM-анализ говорит, что покупатели категории:
    - R - сколько дней прошло с последней покупки клиента: 1 - давно, 3 - недавно
    - F - как клиент часто покупает: 1 - редко, 3 - часто
    - M - сколько прибыли приносит клиент: 1 - мало, 3 много
    Другими словами, следует присмотреться к клиентам 133 и 233, чтобы замотивировать их чаще ходить за покупками. Например, сделать персональные предложения.
    
    Клиентов 313 и 323 также можно замотивировать ходить чаще. Например, использовать рассылку уведомлений о временных акциях на товары.
    
    Кроме того, можно обратить внимание на клиентов 222, чтобы вывести их хотя бы в одну из групп 322, 232 или 223. Подойдут и персональные предложения, и уведомления об акциях.